In [1]:
import numpy as np
import pandas as pd
from pprint import pprint
from random import seed
from random import randrange

df = pd.read_csv("Assignement1-2b.csv")
data = df.values
column_names=df.columns
training_data=df.values
df.head()

,Height,Weight,Age,Gender
0,1.798398,72.545354,29,W
1,1.791747,78.826072,30,W
2,1.768286,69.288869,28,W
3,1.794980,88.006699,22,M
4,1.730018,71.812933,29,W


In [2]:
def purity(dataset):
  classes=set(dataset[:,-1])
  classes=np.array(list(classes))
  # print(classes)
  if len(classes)==1:
    return True
  return False

In [3]:
#checking purity of class
# purity(df[df['Gender']=='W'].values)
# # df[df['Gender']=='W'].values
def classify_treedata(dataset):
  classes=dataset[:,-1]
  unique_classes,count=np.unique(classes,return_counts=True)
  # print(np.unique(classes,return_counts=True))
  classification=unique_classes[np.argmax(count)]
  # print(classification)
  return classification
# classify_treedata(df[(df.Age>30) & (df.Weight>90)].values)

In [4]:
#get the splpit data left and right to decission node
def get_splits(data):
  splits={}
  _,columns=data.shape
  for column in range(columns-1): 
    splits[column]=[]
    values=data[:,column]
    unique_values=np.sort(np.unique(values))
    # print(len(unique_values))
    for index in range(len(unique_values)):
      if(index!=0):
        splits[column].append((unique_values[index-1]+unique_values[index])/2)
  # print(len(splits[2]))
  return splits

In [5]:
def get_splitdata(dataset, column, value):
  split_data=dataset[:,column]
  data_left=dataset[split_data<=value]
  data_right=dataset[split_data>value]
  return data_left,data_right

In [6]:
def entropy_cal(dataset):
  classes=dataset[:,-1]
  _,count=np.unique(classes,return_counts=True)
  prob=count/sum(count)
  entropy=sum(prob * - np.log2(prob))
  return entropy

In [7]:
def total_entropy(data_left, data_right):
  prob_left=len(data_left)/(len(data_left)+len(data_right))
  prob_right=len(data_right)/(len(data_left)+len(data_right))
  total_entropy= (prob_left *entropy_cal(data_left))+ (prob_right *entropy_cal(data_right))
  return total_entropy 

In [8]:
def information_gain(dataset,data_left,data_right):
  return entropy_cal(dataset)-(total_entropy(data_left,data_right))

In [9]:

def get_best_split_gain(dataset,splits):
  best_infogain=0
  for index in splits:
    for value in splits[index]:
      data_left,data_right=get_splitdata(dataset,index,value)
      # best_column=index
      # best_value=value
      info_gain=information_gain(dataset[:,(index,-1)],data_left,data_right)
      if info_gain>best_infogain:
        best_infogain=info_gain
        best_column=index
        best_value=value
  return best_column,best_value

In [10]:

def DecissionTreeClassification(data,min_samples, max_depth,iteration=0):
    if (purity(data)) or (len(data) < min_samples) or (iteration == max_depth):
        classification = classify_treedata(data)
        return classification   
    else:    
        iteration += 1
        splits = get_splits(data)
        best_column, best_value = get_best_split_gain(data, splits)
        data_left, data_right = get_splitdata(data, best_column, best_value)
        # decissionrule=print(f'{column_names[best_column]} <= {best_value}')
        decissionrule=""+str(column_names[best_column])+" <= " + str(best_value) 
        sub_tree = {decissionrule: []}
        left_subtree = DecissionTreeClassification(data_left, min_samples, max_depth,iteration)
        right_subtree = DecissionTreeClassification(data_right, min_samples, max_depth,iteration)
        sub_tree[decissionrule].append(left_subtree)
        sub_tree[decissionrule].append(right_subtree)
        return sub_tree

In [11]:
def predict_gender(test_data,tree):
  # DecissionTreeClassification(training_data,max_depth)
  decissionrule = list(tree.keys())[0]
  #split based on spaces
  feature_name, comparison_operator, value = decissionrule.split()
  if test_data[feature_name] <= float(value):
    answer = tree[decissionrule][0]
  else:
    answer = tree[decissionrule][1]
  #if not dictionary we have reached the classification
  if not isinstance(answer, dict):
      return answer    
  else:
      sub_tree = answer
      return predict_gender(test_data, sub_tree)

In [12]:
#generating samples for bagging
def subsample(dataset,bootstrap_size):
	sample = list()
	n_sample = len(dataset)*bootstrap_size
	while len(sample) < n_sample:
		index = randrange(len(dataset))
		sample.append(dataset[index])
	return np.array(sample)

In [13]:
def bagging(train,min_size,max_depth,sample_size,n_trees):
  trees=list()
  for i in range(n_trees):
    sample=subsample(train,sample_size)
    tree=DecissionTreeClassification(sample,min_size,max_depth)
    trees.append(tree)
  return trees

In [14]:
def bagging_prediction(trees,row):
  predictions = [predict_gender(row,tree) for tree in trees]
  return max(set(predictions), key=predictions.count)


In [15]:
def accuracy(trees,test):
  correct=0
  wrong=0
  for i in range(len(test)):
    predict=bagging_prediction(trees,test.iloc[i])
    if(test.iloc[i].Gender==predict):
      correct+=1
    else:
      wrong+=1
  return (correct/(correct+wrong))*100  

In [17]:
train_data=df[:90]
test_data=df[90:]
seed(2)
print('For Depth 4')
for baggings in [10,50,100]:
  trees=bagging(train_data.values,2,4,0.5,baggings)
  accu=accuracy(trees,test_data)
  error=100-accu
  print('--------------------------------------------')
  # print(len(trees))
  print(f'Accuracy for Bagging {baggings} is {accu}')
  print(f'Error rate for Bagging {baggings} is {error}')
  print('---------------------------------------------')

For Depth 4
--------------------------------------------
Accuracy for Bagging 10 is 63.33333333333333
Error rate for Bagging 10 is 36.66666666666667
---------------------------------------------
--------------------------------------------
Accuracy for Bagging 50 is 70.0
Error rate for Bagging 50 is 30.0
---------------------------------------------
--------------------------------------------
Accuracy for Bagging 100 is 73.33333333333333
Error rate for Bagging 100 is 26.66666666666667
---------------------------------------------
